In [350]:
import pandas as pd
import numpy as np
import re
import math
import xlrd
import datetime
import pprint

In [351]:
def convertDate(date):
    date = '2019-' + date[0:-2] + '-' + date[-2:]
    return date

#date = '2019-' + test_date[-4:].replace('/', '-')

In [352]:
# Turn a hh:mm:ss string to number of seconds
# hh: piece is optional

HHMMSS_PAT = re.compile("(\d+)??:??(\d+):(\d+)$")
def hhmmss_to_seconds(s):
    m = HHMMSS_PAT.match(s)
    g = m.groups()
    s = 0
    n = len(g)
    for i in range(n):
        if g[n-i-1] is not None:
            s += 60**i * int(g[n-i-1])
    return s

In [353]:
def convertTime(time):
    if time == time:
        converted = []
        time = time.replace(" ", "")
        for item in time.split(','):
            time_to_convert = item.split('/')[0]
            converted.append(hhmmss_to_seconds(time_to_convert))
    else:
        converted = []
    return converted

In [354]:
def parseSheet(test_csv, date, position, mouse_labels):
    test_dict = {}
    log = []
    
    #Check that date location is correct
    try:
        isinstance(test_csv.columns.values[1], datetime.datetime)
    except:
            return test_dict, ['Date (or file more generally) is possibly not formatted correctly']
    
    #Check that Video information is in expected location
    try:
        isinstance(test_csv.loc[2][0], str)
    except:
            return test_dict, ['File is possibly not formatted correctly']
    
    #Parse excel sheets
    for item in test_csv.iterrows():
        if item[0]>3 and item[1][0] is np.nan:       #An empty row indicates end of confident annotations in file
            break
        elif item[0]>3:
            try:          
                seizures = {}
                date_dict = {}
                pos_dict = {}
                for i, key in enumerate(mouse_labels):
                    seizures[key] = convertTime(item[1][i+2])
                date_dict[date] = seizures
                pos_dict[position] = date_dict
                test_dict[item[1][0]] = pos_dict
            except Exception as error:
                    log_key = item[1][0] + '_' + 'line' + str(item[0] + 2) + ',  ' + str(error)
                    log.append(log_key)
                    print('There was an issue with the annotation for sheet %s, movie %s' % (date, item[1][0]))
                
    return test_dict, log

In [355]:
#Annotation spreadsheet params

params_dict = {
    
    '3': (('66-2', '70-3', '72-2'), '/Users/mikey/Downloads/Folder3.xlsx'),
    '4': (('65-3', '74-2','71-2', '65-1'), '/Users/mikey/Downloads/Folder4.xlsx'),
    '6 up': (('74-1', '68-2', '65-2', '73-2', '70-1', '73-3'), '/Users/mikey/Downloads/6UP.xlsx'),
    '6 down': (('73-1', '75-2', '66-1', '69-3', '70-2', '71-1'), '/Users/mikey/Downloads/6down.xlsx')
    
}

In [356]:
#Extract annotations for all videos
all_sheets = {}
error_log = {'3':{}, '4':{}, '6 up':{}, '6 down':{}}

for position, value in params_dict.items():
    filename = value[1]
    mouse_labels = value[0]
    test_xls = pandas.ExcelFile(filename)
    for sheet in test_xls.sheet_names:
        this_sheet = pandas.read_excel(test_xls, sheet_name=sheet)
        date = convertDate(sheet)
        new_dict, log = parseSheet(this_sheet, date, position, mouse_labels)
        if new_dict:
            all_sheets.update(new_dict)
        if log:
            error_log[position].update({date:log})


In [357]:
%pprint
pprint.pprint(error_log)

Pretty printing has been turned OFF
{'3': {},
 '4': {},
 '6 down': {'2019-121-19': ['File is possibly not formatted correctly'],
            '2019-213-19': ['File is possibly not formatted correctly']},
 '6 up': {'2019--Q': ['File is possibly not formatted correctly']}}


In [358]:
#Export json 
import json
with open('seizure_annotations_v4.json', 'w') as outfile:
    json.dump(all_sheets, outfile)